In [71]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
import math
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input director
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [72]:
import warnings
warnings.filterwarnings('ignore')

In [128]:
# Load data khi lấy từ lớp full connected
features_train = np.load("/kaggle/input/ge-features/features_train.npy")

In [129]:
features_train.shape

(8502, 6400)

In [130]:
# # Merge train and valid's label
train_label_df = pd.read_csv("/kaggle/input/ge-muse-split/train_label.csv")
valid_label_df = pd.read_csv("/kaggle/input/ge-muse-split/val_label.csv")
label_df = pd.concat((train_label_df, valid_label_df), axis = 0)
label_df.shape

(8502, 7)

In [131]:
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()

label_pca = label_df["Rhythm"]
label_enc = lb.fit_transform(label_pca)
label_enc

array([0, 1, 1, ..., 2, 1, 2])

In [132]:
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(features_train)
pca.explained_variance_ratio_

array([1.9842398e-01, 1.2980640e-01, 9.6689872e-02, ..., 2.9636854e-07,
       2.9489220e-07, 2.8896790e-07], dtype=float32)

In [133]:
# Tìm số n để giữ lại 99% lượng thông tin
explained_variance = np.cumsum(pca.explained_variance_ratio_)
num_components = np.argmax(explained_variance >= 0.99) + 1
print("Số components để giữ lại 99% lượng thông tin: ", num_components)

Số components để giữ lại 99% lượng thông tin:  3967


In [134]:
pca = PCA(n_components = num_components)
features_pca_df = pca.fit_transform(features_train)
features_pca_df.shape

(8502, 3967)

In [136]:
X_train, y_train = features_pca_df, label_enc
print(f'X_train: {X_train.shape} \ny_train: {y_train.shape}')

X_train: (8502, 3967) 
y_train: (8502,)


In [ ]:
# Dùng SVM để phân loại
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from xgboost import XGBClassifier

clf = KNeighborsClassifier()
param_grid = {
    'n_neighbors': np.arange(1, 31)
}
grid = GridSearchCV(clf, param_grid, refit = True, verbose = 3) 
  
grid.fit(X_train, y_train) 

In [139]:
print(grid.best_params_ )

{'C': 0.01, 'gamma': 0.001, 'kernel': 'linear'}


In [140]:
print(grid.best_estimator_)

SVC(C=0.01, gamma=0.001, kernel='linear')


In [141]:
# # Load tập data test sau khi qua lớp full connected
X_test = np.load("/kaggle/input/ge-features/features_test.npy")
X_test.shape

(2126, 6400)

In [142]:
X_test = pca.transform(X_test)
X_test.shape

(2126, 3967)

In [143]:
y_test = lb.fit_transform(pd.read_csv("/kaggle/input/ge-muse-split/test_label.csv")["Rhythm"].values)
y_test.shape

(2126,)

In [145]:
y_pred = grid.predict(X_test)

In [146]:
y_pred.shape

(2126,)

In [ ]:
from sklearn.metrics import classification_report

print("Kết quả khi áp dụng PCA:")
print(classification_report(y_test, y_pred, target_names= lb.classes_))

In [ ]:
result = classification_report(y_test, y_pred, target_names= lb.classes_, output_dict = True)
result_df = pd.DataFrame(result).transpose()
result_df

In [149]:
result_df.to_csv('./result.csv', index=True)